In [5]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

df = pd.read_csv('yelp_labelled.txt', names=['sentence', 'label'], sep='\t')
df.shape

(1000, 2)

In [6]:
df.head()

,sentence,label
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [7]:
# convert to lowercase
df['sentence'] = df['sentence'].str.lower()

In [10]:
# remove stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
stop = set(stopwords.words('english'))
df['sentence'] = df['sentence'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop)]))
df.head()

,sentence,label
0,wow... loved place.,1
1,crust good.,0
2,tasty texture nasty.,0
3,stopped late may bank holiday rick steve recom...,1
4,selection menu great prices.,1


In [15]:
#tokenisasi
#metadata: word index hasil tokenisasi
vocab_size = 2000
oov_tok = "<OOV>"
filt = '!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ' #remove symbols

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok, filters = filt)
tokenizer.fit_on_texts(df['sentence'].values)

word2index = tokenizer.word_index
print(len(word2index))

1998


In [17]:
#ubah ke JSON
#serialisasi variabel agar dapat diunduh komputer
import json

with open('word2index.json', 'w') as fp:
    json.dump(word2index, fp)

In [14]:
max_length =  max(len(values.split()) for i, values in enumerate(df['sentence']))
max_length

18

In [18]:
trunc_type='post'

all_seq = tokenizer.texts_to_sequences(df['sentence'].values)
all_padded = pad_sequences(all_seq, maxlen = max_length, padding = trunc_type)
all_padded.shape

(1000, 18)

In [19]:
# split train and test sets
from sklearn.model_selection import train_test_split

X = all_padded
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(800, 18) (800,)
(200, 18) (200,)


In [20]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim= vocab_size, output_dim=16, input_length= max_length),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 18, 16)            32000     
_________________________________________________________________
lstm (LSTM)                  (None, 64)                20736     
_________________________________________________________________
dense (Dense)                (None, 24)                1560      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 54,321
Trainable params: 54,321
Non-trainable params: 0
_________________________________________________________________


In [21]:
num_epochs = 30
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test))

Epoch 1/30
25/25 [==============================] - 3s 33ms/step - loss: 0.6940 - accuracy: 0.5050 - val_loss: 0.6944 - val_accuracy: 0.4800
Epoch 2/30
25/25 [==============================] - 0s 12ms/step - loss: 0.6930 - accuracy: 0.5213 - val_loss: 0.6929 - val_accuracy: 0.4800
Epoch 3/30
25/25 [==============================] - 0s 13ms/step - loss: 0.6254 - accuracy: 0.6562 - val_loss: 0.5671 - val_accuracy: 0.7050
Epoch 4/30
25/25 [==============================] - 0s 12ms/step - loss: 0.2683 - accuracy: 0.9112 - val_loss: 0.6328 - val_accuracy: 0.7250
Epoch 5/30
25/25 [==============================] - 0s 12ms/step - loss: 0.1128 - accuracy: 0.9638 - val_loss: 0.8937 - val_accuracy: 0.7350
Epoch 6/30
25/25 [==============================] - 0s 13ms/step - loss: 0.0754 - accuracy: 0.9800 - val_loss: 0.9480 - val_accuracy: 0.7250
Epoch 7/30
25/25 [==============================] - 0s 13ms/step - loss: 0.0523 - accuracy: 0.9837 - val_loss: 0.9492 - val_accuracy: 0.7350
Epoch 8/30
25

In [22]:
!pip install tensorflowjs

     |████████████████████████████████| 64 kB 2.0 MB/s 


In [23]:
saved_model_path = '/content/mymodel/'
tf.saved_model.save(model, saved_model_path)

INFO:tensorflow:Assets written to: /content/mymodel/assets


INFO:tensorflow:Assets written to: /content/mymodel/assets


In [24]:
!tensorflowjs_converter \
  --input_format=tf_saved_model \
  /content/mymodel/ \
  /content/modeltfjs

2021-10-14 06:39:56.350029: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-14 06:39:56.350097: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (e446032863da): /proc/driver/nvidia/version does not exist
2021-10-14 06:39:58.102911: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2021-10-14 06:39:58.103190: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-10-14 06:39:58.112892: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: Graph size after: 251 nodes (239), 348 edges (336), time = 4.898ms.
  function_optimizer: function_optimizer did nothing. time = 0.087ms.

2021-10-14 06:39:58.316412: I tensorflow/core/grappler/optimizers/meta_optim